<a href="https://colab.research.google.com/github/yongsun-yoon/deep-learning-paper-implementation/blob/main/03-natural-language-process/PatternRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PatternRank

## 0. Info

### paper
* title: PatternRank: Leveraging Pretrained Language Models and Part of Speech for Unsupervised Keyphrase Extraction
* author: Tim Schopf et al.
* url: https://arxiv.org/abs/2210.05245

### feats
* data: KorQuAD
* sbert: Huffon/sentence-klue-roberta-base, [link](https://huggingface.co/Huffon/sentence-klue-roberta-base)

### refs
* https://github.com/TimSchopf/KeyphraseVectorizers

## 1. Setup

In [ ]:
import nltk
import spacy
import numpy as np
from kiwipiepy import Kiwi

import torch
from sentence_transformers import SentenceTransformer, util
from datasets import load_dataset

## 2. Prepare

In [ ]:
tagger = Kiwi()

In [ ]:
sbert = SentenceTransformer("Huffon/sentence-klue-roberta-base", device='cuda:0')

No sentence-transformers model found with name /root/.cache/torch/sentence_transformers/Huffon_sentence-klue-roberta-base. Creating a new one with MEAN pooling.


In [ ]:
data = load_dataset('KETI-AIR/korquad', 'v1.0')
data = data['train']

Found cached dataset korquad (/root/.cache/huggingface/datasets/KETI-AIR___korquad/v1.0/1.0.0/2f25bce677d9a1757c546a0db238083efafcb3b2c254b5883136455d7133abe1)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
data = data.to_pandas()
data = data['context'].drop_duplicates().tolist()

## 3. Run

In [ ]:
def extract_keyphrases(text, tagger, sbert, pattern='<N.*>+', k=5, max_length=768):
    tokens = tagger.tokenize(text)
    pos_tuples = [(t.form, t.tag) for t in tokens]
    
    parser = nltk.RegexpParser('CHUNK: {(' + pattern + ')}')
    tree = parser.parse(pos_tuples)
    
    candidates = []
    for subtree in tree.subtrees(filter=lambda tuple: tuple.label() == 'CHUNK'):
        cand = ' '.join([i[0] for i in subtree.leaves()])
        cand = cand.strip()
        candidates.append(cand)
    candidates = list(set(candidates))
    
    text = text[:max_length]
    embeds = sbert.encode([text] + candidates)
    scores = util.pytorch_cos_sim(embeds[:1], embeds[1:])[0].numpy()
    ranks = np.argsort(-scores)
    
    keyphrases = []
    for r in ranks:
        candidate = candidates[r]
        score = scores[r]
        if candidate not in keyphrases:
            keyphrases.append({'keyphrase': candidate, 'score': score})
        if len(keyphrases) == k:
            break
            
    return keyphrases

In [ ]:
text = data[0]
text

'1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.'

In [ ]:
keyphrases = extract_keyphrases(text, tagger, sbert)
keyphrases

[{'keyphrase': '교향곡 작곡', 'score': 0.34518024},
 {'keyphrase': '시기 바그너', 'score': 0.2832832},
 {'keyphrase': '합창 교향곡 조성', 'score': 0.2767795},
 {'keyphrase': '파리 음악원 관현악단', 'score': 0.24829283},
 {'keyphrase': '교향곡', 'score': 0.23332232}]